In [ ]:
#imports
import pandas as pd 
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import torch
import torch.nn as nn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [ ]:
os.path.abspath(os.getcwd())

df_standards_warm = pd.read_csv("../data/standards_warm/Data_28_5__15_41_20_0_zu_2500.csv")  
df_warm_standards = pd.read_csv("../data/warm_standards_hamilton/Data_9_6__16_12_35_0_zu_2500.csv")

#todo ask what is valueRAW and what is value? my guess is that its the same data but just scaled somehow (see plot)


In [ ]:
df_warm_standards.head

In [ ]:
df_standards_warm.head

In [ ]:
time = df_standards_warm["Time"]
values = df_standards_warm["Value"]
values_raw = df_standards_warm["ValueRAW"]

In [ ]:
#plot time to values
plt.plot(values,time)
plt.yticks(range(0, len(time), 100))
plt.show()

In [ ]:
#plot time to valuesRAW

plt.plot(values_raw,time)
plt.yticks(range(0, len(time), 100))
plt.show()

In [ ]:
#get data

with open("../data/data_max/value_target.csv", "w") as f:
    f.write("")

with open("../data/data_max/value_target.csv", "a") as f:
    header = "target"
    for i in range(1, 1001):
        header = header + ",value_" + str(i)
    f.write(header + "\n")

#iter over every file from a folder
for filename in os.listdir("../data/standards_warm/"):
    #open file
    df_standards_warm = pd.read_csv("../data/standards_warm/"+filename)  
    #get the number which appears at the end between _...csv
    number = filename.split("_")[-1].split(".")[0]
    #get the values from the column value
    values = df_standards_warm["Value"]
    #create a new file with the name data_max/file
    with open("../data/data_max/value_target.csv", "a") as f:
        #write the number which appears at the end between _...csv in a row of data/data_max/file
        f.write(number)
        #write a comma
        f.write(",")
        #write the values from the column value in a row of data/data_max/file
        f.write(",".join(str(x) for x in values))
        #write a new line
        f.write("\n")
        
#iter over every file from a folder
for filename in os.listdir("../data/warm_standards_hamilton"):
    #open file
    df_standards_warm = pd.read_csv("../data/warm_standards_hamilton/"+filename)  
    #get the number which appears at the end between _...csv
    number = filename.split("_")[-1].split(".")[0]
    #get the values from the column value
    values = df_standards_warm["Value"]
    #create a new file with the name data_max/file
    with open("../data/data_max/value_target.csv", "a") as f:
        #write the number which appears at the end between _...csv in a row of data/data_max/file
        f.write(number)
        #write a comma
        f.write(",")
        #write the values from the column value in a row of data/data_max/file
        f.write(",".join(str(x) for x in values))
        #write a new line
        f.write("\n")

In [ ]:
df_value_target = pd.read_csv("../data/data_max/value_target.csv")

test = df_value_target.tail(3)

train = df_value_target.drop(df_value_target.tail(3).index)

test_x = test.drop("target", axis=1)

test_y = test["target"]

train_x = train.drop("target", axis=1)

train_y = train["target"]

#later for creating onnx file necessary
input_names = list(train_x)
output_names = list(train_y)

# Regression

In [ ]:
train_x = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)
test_x = pd.DataFrame(test_x)
test_y = pd.DataFrame(test_y)

scalerX = StandardScaler().fit(train_x)
scalerY = StandardScaler().fit(train_y)
train_x = scalerX.transform(train_x)
train_y = scalerY.transform(train_y)
test_x = scalerX.transform(test_x)
test_y = scalerY.transform(test_y)

model = LinearRegression().fit(train_x, train_y)

predictions = model.predict(test_x)

#print(predictions)
print(test_y)
#convert tensor to numpy
test_y = np.array(test_y)

#calculate MAE 
mae = sum(abs(predictions - test_y))/len(predictions)
print(mae)

# Simple Neural Network

In [ ]:
test_x = pd.DataFrame(test_x)
test_y = pd.DataFrame(test_y)
train_x = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)

#convert data to tensors
test_x = torch.tensor(test_x.to_numpy()).float()
test_y = torch.tensor(test_y.to_numpy()).float()
train_x = torch.tensor(train_x.to_numpy()).float()
train_y = torch.tensor(train_y.to_numpy()).float()

#create a simple neural network with pytroch
model = torch.nn.Sequential(
    nn.Linear(1000, 100),
    nn.ReLU(),
    nn.Linear(100, 10),
    nn.ReLU(),
    nn.Linear(10, 1)
)

criterion = torch.nn.MSELoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

for epoch in range(1000):
    y_pred = model(train_x)
    loss = criterion(y_pred, train_y.float())
    print("epoch: ", epoch, "loss: ", loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
#cacluate predictions
pred_y = model(test_x.float())

#calculate MAE
mae = sum(abs(pred_y - test_y))/len(pred_y)
#convert tensor to array 
mae = mae.detach().numpy()
print("MAE: " + str(mae[0]))

#rescale data
pred_y = scalerY.inverse_transform(pred_y.detach().numpy())
test_y = scalerY.inverse_transform(test_y.detach().numpy())

#print(pred_y)
#print(test_y)



# Cross validation simple neural network

In [ ]:
def scaler(train_x, train_y, test_x, test_y):
    """Scale data to standardization

    
    Returns:
        df: scaled data
    """
    test_x = pd.DataFrame(test_x)
    test_y = pd.DataFrame(test_y)
    train_x = pd.DataFrame(train_x)
    train_y = pd.DataFrame(train_y)
    
    scalerX = StandardScaler().fit(train_x)
    scalerY = StandardScaler().fit(train_y)
    train_x = scalerX.transform(train_x)
    train_y = scalerY.transform(train_y)
    test_x = scalerX.transform(test_x)
    test_y = scalerY.transform(test_y)
    
    test_x = pd.DataFrame(test_x)
    test_y = pd.DataFrame(test_y)
    train_x = pd.DataFrame(train_x)
    train_y = pd.DataFrame(train_y)

    test_x = torch.tensor(test_x.to_numpy()).float()
    test_y = torch.tensor(test_y.to_numpy()).float()
    train_x = torch.tensor(train_x.to_numpy()).float()
    train_y = torch.tensor(train_y.to_numpy()).float()
    
    return train_x, train_y, test_x, test_y
    
def model_standard(train_x, train_y, test_x, test_y):
    """ML modell for glucose prediction

    

    Returns:
        int: mae of the model trained with data split based on cross validation
    """
    train_x, train_y, test_x, test_y = scaler(train_x, train_y, test_x, test_y)
    
    model = torch.nn.Sequential(
    nn.Linear(1000, 100),
    nn.ReLU(),
    nn.Linear(100, 10),
    nn.ReLU(),
    nn.Linear(10, 1)
    )

    criterion = torch.nn.MSELoss()

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

    for epoch in range(1000):
        y_pred = model(train_x)
        loss = criterion(y_pred, train_y.float())
        #print("epoch: ", epoch, "loss: ", loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    pred_y = model(test_x.float())
    mae = sum(abs(pred_y - test_y))/len(pred_y)
    mae = str(mae.detach().numpy()[0])
    return mae
    

def data_split(data, iteration_split, number_splits, check_for_last_iteration):
    """Splits data into train and test data.

    Args:
        data (df): df with all the data
        iteration_split (int): current batch step of cross validation
        number_splits ([type]): number of batches
        check_for_last_iteration (boolean): check if it is the last batch

    Returns:
        df: train and test data
    """
    if check_for_last_iteration == True:
        test = data.iloc[iteration_split * number_splits:]
        train = data.drop(data.iloc[iteration_split * number_splits:].index)
    else:
        test = data.iloc[iteration_split * number_splits : (iteration_split * number_splits + number_splits)]
        train = data.drop(data.iloc[iteration_split * number_splits : (iteration_split * number_splits + number_splits)].index)    
    
    test_x = test.drop("target", axis=1)

    test_y = test["target"]

    train_x = train.drop("target", axis=1)

    train_y = train["target"]
    
    return train_x, train_y, test_x, test_y

def cross_validation(model, data, number_splits):
    list_mae = []
    len_data = len(data)
    splits = len_data / number_splits
    split = int(splits)
    for i in range(number_splits):
        check_for_last_iteration = False
        if i == number_splits - 1:
            check_for_last_iteration = True
        train_x, train_y, test_x, test_y = data_split(data, i, split, check_for_last_iteration)
        model_result = model_standard(train_x, train_y, test_x, test_y)
        list_mae.append(model_result)
    print(list_mae)

cross_validation_result = cross_validation(model, df_value_target,5)
    

# Backtesting

In [ ]:
#backtesting is if I take 800 instead of 1000 x features for example and than i try to predict the next 200 values



# Netron architecture visualization

In [ ]:
#visualize model structure with netron
model.eval()
input_names = ['Values']
output_names = ['Targets']

torch.onnx.export(model,
                  train_x, 
                  "../model_structure/simple_nn.onnx", 
                  input_names=input_names, 
                  output_names=output_names,
                  )    

# AutoML 

In [ ]:
#H2OAutoML
#Auto-Pytorch from Uni Freiburg
#Uber Ludwig

## Auto-PyTorch

In [ ]:
#i again start with loading the fucking data would be great to have a method which I can call all the time 
import h2o
h2o.init()
h2o.demo("glm")

In [ ]:
#uninstall glucose venv -
#create glucose venv - 
#install pyton 3.11 with brew and try to get h2o running 
#create venv in brew python - 
#check packages --> needs to be empty - 
#run first cell of this notebook - 
#install required python - 
#install numpy and import numpy in a cell and check if i dont get an error - 
#install h20 
#run a demo succesfully

#create virtual env in commandline
#activate virtual env

In [ ]:
import numpy 

In [ ]:
import numpy as np

In [1]:
import h2o
h2o.init()
h2o.demo("glm")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...


CalledProcessError: Command '['/usr/bin/java', '-version']' returned non-zero exit status 1.